In [21]:
import pandas as pd
from datasets import load_dataset

# Load the Bengali subset of the dataset
dataset = load_dataset('csebuetnlp/xlsum', 'bengali')

# Convert to pandas dataframe
df = dataset['test'].to_pandas()  # Assuming you want the training split

df = df.head(512) # we choose first 512 samples for evaluation

# Display the dataframe
df.head()

,id,url,title,summary,text
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা..."
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...


In [22]:
len(df)

512

In [11]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

In [13]:
LOCATION = "us-central1"

In [14]:
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.5-flash")

def get_gemini_output(prompt = "", max_out_tokens = 128, temp = 0.0, top_p = 0.9, top_k = 10):
    responses = model.generate_content(
        contents=prompt,
        generation_config={
            "max_output_tokens": max_out_tokens,
            "temperature": temp,
            "top_p": top_p,
            "top_k": top_k
        },
    stream=True,
    )

    response_list = []
    for response in responses:
        try:
            response_list.append(response.text)
        except IndexError:
            response_list.append("")
            continue
    prompt_response = " ".join(response_list)
    return prompt_response

In [15]:
get_gemini_output("বাংলাদেশের ৫ জন কবির নাম লিস্ট আকারে লিখ।")

'বা ংলাদেশের ৫ জন কবির নাম লিস্ট  আকারে:\n\n1. কাজী নজরুল ইস লাম\n2. রবীন্দ্রনাথ ঠাকুর\n3. মাইকেল মধুসূদন দত্ত\n 4. সুকান্ত ভট্টাচার্য\n5. জীবনানন্দ দাশ \n '

In [23]:
zero_shot = """আপনাকে একটি দীর্ঘ বাংলা বিবরণ দেওয়া হয়েছে। আপনাকে এই বিবরণের ১ বা ২ বাক্যে সারমর্ম লিখতে হবে।

"""

### Three-shot examples

We took the examples from XL-sum dataset (not in the eval.) which are the shortest.

In [24]:
three_shot = """আপনাকে একটি দীর্ঘ বাংলা বিবরণ দেওয়া হয়েছে। আপনাকে এই বিবরণের ১ বা ২ বাক্যে সারমর্ম লিখতে হবে।

### উদাহরণ ১
### বিবরণ:
আলজেরিয়ায় পরীক্ষার হল স্কুলের ছাত্রছাত্রীরা যাতে পরীক্ষার হলে নকল করতে না পারে কিম্বা আগেভাগেই প্রশ্ন না পেতে না পারে সেজন্যে কর্তৃপক্ষ সাময়িকভাবে দেশটিতে সোশাল মিডিয়া বন্ধ করে দিয়েছে। এর আগে স্কুলের পরীক্ষায় অনলাইনের মাধ্যমে বহু প্রশ্ন ফাঁস হয়ে গেলে দেশটিতে কর্তৃপক্ষের তীব্র সমালোচনা হয়। তখন অর্ধেকেরও বেশি শিক্ষার্থীকে দ্বিতীয়বার পরীক্ষায় বসত হয়েছে। সংবাদ মাধ্যমগুলোর খবরে বলা হচ্ছে, জুন মাসে বহু ছাত্রছাত্রীই পরীক্ষার আগে ফেসবুকে ও অন্যান্য সোশাল মিডিয়া থেকে প্রশ্ন হাতে পেয়ে যায়। এর আগের বছরগুলোতেও দেশটিতে এধরনের ঘটনা ঘটেছে। সরকারি কর্মকর্তারা বলছেন, পরীক্ষার্থীদেরকে নকল ও প্রশ্ন ফাঁসের হাত থেকে রক্ষার জন্যেই সোশাল মিডিয়া ব্লক করে দেওয়ার সিদ্ধান্ত নেওয়া হয়েছে।

### সারমর্ম:
আফ্রিকার দেশ আলজেরিয়ায় পরীক্ষায় নকল ও প্রশ্ন ফাঁস করতে নজিরবিহীন এক উদ্যোগ নিয়েছে।

### উদাহরণ ২
### বিবরণ:
চ্যাম্পিয়ন্স লিগে এক ক্লাবের হয়ে ১০০ গোল করা একমাত্র খেলোয়াড় রোনালদো বুধবার চ্যাম্পিয়ন্স লিগে পিএসজির বিপক্ষে ৩-১এর জয়ে ২ গোল করে এক ক্লাবের হয়ে ১০০ চ্যাম্পিয়ন্স লিগ গোল করা একমাত্র খেলোয়াড় হয়েছেন রেয়াল মাদ্রিদের ক্রিশ্চিয়ানো রোনালদো। এই তালিকায় দ্বিতীয় স্থানে রয়েছেন তাঁর চেয়ে ৪ গোল কম করা লিওনেল মেসি। রিয়াল মাদ্রিদের হয়ে ১০০ গোলের পর রোনালদো চ্যাম্পিয়ন্স লিগে রোনালদোর আরো রেকর্ড রেয়াল মাদ্রিদে যোগ দেয়ার আগে ম্যানচেস্টার ইউনাইটেডের হয়ে করা ১৫টি গোল মিলিয়ে চ্যাম্পিয়ন্স লিগে রোনালদোর মোট গোল ১১৬টি। যা দ্বিতীয় স্থানে থাকা মেসির চেয়ে ১৯টি বেশী। চ্যাম্পিয়ন্স লিগের ইতিহাসে এই টুর্নামেন্টে খেলা ১৩৭টি ক্লাবের ১১৮টির মোট গোলসংখ্যা রোনালদোর একার গোলের চেয়ে কম।

### সারমর্ম:
১০০ গোলের ক্লাবে একমাত্র সদস্য - ক্রিশ্চিয়ানো রোনালদো।

### উদাহরণ ৩
### বিবরণ:
বৈরুতের বন্দরের নিকটবর্তী অঞ্চল সবচেয়ে বেশি ক্ষতিগ্রস্ত হয় বিস্ফোরণে এমনও খবর পাওয়া যাচ্ছে যে কয়েক'শো কিলোমিটার দূরের সাইপ্রাসেও ঐ বিস্ফোরণের আঁচ পাওয়া গেছে। কর্মকর্তারা বলছেন এই বিস্ফোরণ দুর্ঘটনা। পরিকল্পিতভাবে এই বিস্ফোরণ ঘটানো হয়নি। তারা বলছেন গুদামে ছয় বছর ধরে মজুত রাখা অত্যন্ত বিপদজনক বিস্ফোরক থেকে এই দুর্ঘটনা ঘটেছে। তবে ঠিক কী কারণে এই বিস্ফোরণ ঘটেছে, তা পরিস্কার নয়। ক্ষতিগ্রস্থ ভবনের সামনে লেবাননের 'লেবানিজ এমিগ্র্যান্ট' স্ট্যাচু বহু ঘরবাড়ি ধ্বংস হয়ে গেছে বিস্ফোরণে বিস্ফোরণে পর বৈরুতের আকাশে ধোঁয়া আহত এক ব্যক্তিকে সরিয়ে নিয়ে যাচ্ছেন উদ্ধারকারীরা বিস্ফোরণে আহত হয়েছেন হাজার হাজার মানুষ স্থানীয় সংবাদমাধ্যমের খবরে ধ্বংসস্তুপের নিচে মানুষকে আটকা পড়ে থাকতে দেখা গেছে। বিস্ফোরণের পর রাস্তাগুলোও যেন ধ্বংসস্তূপে পরিণত হয় বিস্ফোরণের স্থান থেকে নির্গত হচ্ছে ধোঁয়া সব ছবির কপিরাইট রয়েছে

### সারমর্ম:
বিশাল এক বিস্ফোরণ ব্যাপক ধ্বংসযজ্ঞ চালিয়েছে লেবাননের রাজধানী বৈরুতে। বহু মানুষ মারা গেছে, আহত হয়েছে কয়েক হাজারের বেশি।

### আপনার কাজ

"""

In [25]:
chain_of_thought = """আপনাকে একটি দীর্ঘ বাংলা বিবরণ দেওয়া হয়েছে। আপনাকে এই বিবরণের ১ বা ২ বাক্যে সারমর্ম লিখতে হবে। সারমর্ম লিখতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: মূল বিষয়বস্তু বোঝা
বিবরণটি পড়ুন এবং এর মূল বিষয় বা ঘটনাটি বোঝার চেষ্টা করুন। প্রধান তথ্য বা ঘটনা শনাক্ত করুন যা বিবরণের সারমর্ম হতে পারে।

### ধাপ ২: প্রধান তথ্য চিহ্নিত করা
বিবরণের প্রধান তথ্য বা ঘটনার সংক্ষিপ্ত রূপ বের করুন। কোন বিষয়গুলি সবচেয়ে গুরুত্বপূর্ণ তা চিহ্নিত করুন।

### ধাপ ৩: সারমর্ম লেখা
বিবরণের প্রধান তথ্যগুলি ব্যবহার করে ১ বা ২ বাক্যে সারমর্ম লিখুন। এটি সংক্ষিপ্ত এবং সুনির্দিষ্ট হতে হবে।

### উদাহরণ ১
### বিবরণ:
আলজেরিয়ায় পরীক্ষার হল স্কুলের ছাত্রছাত্রীরা যাতে পরীক্ষার হলে নকল করতে না পারে কিম্বা আগেভাগেই প্রশ্ন না পেতে না পারে সেজন্যে কর্তৃপক্ষ সাময়িকভাবে দেশটিতে সোশাল মিডিয়া বন্ধ করে দিয়েছে। এর আগে স্কুলের পরীক্ষায় অনলাইনের মাধ্যমে বহু প্রশ্ন ফাঁস হয়ে গেলে দেশটিতে কর্তৃপক্ষের তীব্র সমালোচনা হয়। তখন অর্ধেকেরও বেশি শিক্ষার্থীকে দ্বিতীয়বার পরীক্ষায় বসত হয়েছে। সংবাদ মাধ্যমগুলোর খবরে বলা হচ্ছে, জুন মাসে বহু ছাত্রছাত্রীই পরীক্ষার আগে ফেসবুকে ও অন্যান্য সোশাল মিডিয়া থেকে প্রশ্ন হাতে পেয়ে যায়। এর আগের বছরগুলোতেও দেশটিতে এধরনের ঘটনা ঘটেছে। সরকারি কর্মকর্তারা বলছেন, পরীক্ষার্থীদেরকে নকল ও প্রশ্ন ফাঁসের হাত থেকে রক্ষার জন্যেই সোশাল মিডিয়া ব্লক করে দেওয়ার সিদ্ধান্ত নেওয়া হয়েছে।

#### চিন্তাধারা:
- মূল বিষয়বস্তু: পরীক্ষার হলে নকল ও প্রশ্ন ফাঁস ঠেকাতে সোশাল মিডিয়া বন্ধ।
- প্রধান তথ্য: নকল ও প্রশ্ন ফাঁস ঠেকাতে আলজেরিয়ায় সোশাল মিডিয়া সাময়িকভাবে বন্ধ।

### সারমর্ম:
আফ্রিকার দেশ আলজেরিয়ায় পরীক্ষায় নকল ও প্রশ্ন ফাঁস করতে নজিরবিহীন এক উদ্যোগ নিয়েছে।

### উদাহরণ ২
### বিবরণ:
চ্যাম্পিয়ন্স লিগে এক ক্লাবের হয়ে ১০০ গোল করা একমাত্র খেলোয়াড় রোনালদো বুধবার চ্যাম্পিয়ন্স লিগে পিএসজির বিপক্ষে ৩-১এর জয়ে ২ গোল করে এক ক্লাবের হয়ে ১০০ চ্যাম্পিয়ন্স লিগ গোল করা একমাত্র খেলোয়াড় হয়েছেন রেয়াল মাদ্রিদের ক্রিশ্চিয়ানো রোনালদো। এই তালিকায় দ্বিতীয় স্থানে রয়েছেন তাঁর চেয়ে ৪ গোল কম করা লিওনেল মেসি। রিয়াল মাদ্রিদের হয়ে ১০০ গোলের পর রোনালদো চ্যাম্পিয়ন্স লিগে রোনালদোর আরো রেকর্ড রেয়াল মাদ্রিদে যোগ দেয়ার আগে ম্যানচেস্টার ইউনাইটেডের হয়ে করা ১৫টি গোল মিলিয়ে চ্যাম্পিয়ন্স লিগে রোনালদোর মোট গোল ১১৬টি। যা দ্বিতীয় স্থানে থাকা মেসির চেয়ে ১৯টি বেশী। চ্যাম্পিয়ন্স লিগের ইতিহাসে এই টুর্নামেন্টে খেলা ১৩৭টি ক্লাবের ১১৮টির মোট গোলসংখ্যা রোনালদোর একার গোলের চেয়ে কম।

#### চিন্তাধারা:
- মূল বিষয়বস্তু: রোনালদোর ১০০ গোল করা।
- প্রধান তথ্য: এক ক্লাবের হয়ে ১০০ গোল করা একমাত্র খেলোয়াড় রোনালদো।

### সারমর্ম:
১০০ গোলের ক্লাবে একমাত্র সদস্য - ক্রিশ্চিয়ানো রোনালদো।

### উদাহরণ ৩
### বিবরণ:
বৈরুতের বন্দরের নিকটবর্তী অঞ্চল সবচেয়ে বেশি ক্ষতিগ্রস্ত হয় বিস্ফোরণে এমনও খবর পাওয়া যাচ্ছে যে কয়েক'শো কিলোমিটার দূরের সাইপ্রাসেও ঐ বিস্ফোরণের আঁচ পাওয়া গেছে। কর্মকর্তারা বলছেন এই বিস্ফোরণ দুর্ঘটনা। পরিকল্পিতভাবে এই বিস্ফোরণ ঘটানো হয়নি। তারা বলছেন গুদামে ছয় বছর ধরে মজুত রাখা অত্যন্ত বিপদজনক বিস্ফোরক থেকে এই দুর্ঘটনা ঘটেছে। তবে ঠিক কী কারণে এই বিস্ফোরণ ঘটেছে, তা পরিস্কার নয়। ক্ষতিগ্রস্থ ভবনের সামনে লেবাননের 'লেবানিজ এমিগ্র্যান্ট' স্ট্যাচু বহু ঘরবাড়ি ধ্বংস হয়ে গেছে বিস্ফোরণে বিস্ফোরণে পর বৈরুতের আকাশে ধোঁয়া আহত এক ব্যক্তিকে সরিয়ে নিয়ে যাচ্ছেন উদ্ধারকারীরা বিস্ফোরণে আহত হয়েছেন হাজার হাজার মানুষ স্থানীয় সংবাদমাধ্যমের খবরে ধ্বংসস্তুপের নিচে মানুষকে আটকা পড়ে থাকতে দেখা গেছে। বিস্ফোরণের পর রাস্তাগুলোও যেন ধ্বংসস্তূপে পরিণত হয় বিস্ফোরণের স্থান থেকে নির্গত হচ্ছে ধোঁয়া সব ছবির কপিরাইট রয়েছে

#### চিন্তাধারা:
- মূল বিষয়বস্তু: বৈরুতের বিস্ফোরণ।
- প্রধান তথ্য: বিস্ফোরণে বৈরুতে ব্যাপক ধ্বংসযজ্ঞ ও বহু মানুষ আহত।

### সারমর্ম:
বিশাল এক বিস্ফোরণ ব্যাপক ধ্বংসযজ্ঞ চালিয়েছে লেবাননের রাজধানী বৈরুতে। বহু মানুষ মারা গেছে, আহত হয়েছে কয়েক হাজারের বেশি।

### আপনার কাজ

"""

### Zero-shot

In [13]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_summary'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gemini_zs_summary_results.parquet')
    start_index = df_progress['predicted_summary'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    text_body = row['text']
    
    prompt_input = zero_shot + " বিবরণ: " + text_body + "\nসারমর্ম:"

    try:
        df.at[i, 'predicted_summary'] = get_gemini_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gemini_zs_summary_results.parquet')

# Save final progress
df.to_parquet('gemini_zs_summary_results.parquet')

  0%|          | 0/512 [00:00<?, ?it/s]

Error at index 18: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.40157047,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.16505408
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.44456476,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.20705435
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.35846534,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.24310295
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probability": "MEDIUM",
   

In [15]:
zs_results = pd.read_parquet("gemini_zs_summary_results.parquet")
zs_results.head()

,id,url,title,summary,text,predicted_summary
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...,চ ীনের জিনজিয়াং প্রদেশে উইগুর ম ুসলিমদের গণহা...
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...",ত ামিলনাডুতে স্থানীয় পণ্যের ব্য বহার নিশ্চিত ...
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...,বা ংলাদেশে ডেঙ্গু রোগের প্রাদুর্ ভাবের কারণে হ...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,ড োনাল্ড ট্রাম্প টুইটারকে আমের িকার ২০২০ নির্ব...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,বা ংলাদেশের উত্তর ও মধ্যাঞ্চলে বন্যার প্রকোপ ...


In [18]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_summary'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gemini_ts_summary_results.parquet')
    start_index = df_progress['predicted_summary'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    text_body = row['text']
    
    prompt_input = three_shot + "### বিবরণ: " + text_body + "\n### সারমর্ম:"

    try:
        df.at[i, 'predicted_summary'] = get_gemini_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gemini_ts_summary_results.parquet')

# Save final progress
df.to_parquet('gemini_ts_summary_results.parquet')

  0%|          | 0/512 [00:00<?, ?it/s]

Error at index 0: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "LOW",
      "probability_score": 0.6311397,
      "severity": "HARM_SEVERITY_HIGH",
      "severity_score": 0.8095773
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.7631309,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.44227514
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.7613611,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.5915463
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "pr

In [19]:
ts_results = pd.read_parquet("gemini_ts_summary_results.parquet")
ts_results.head()

,id,url,title,summary,text,predicted_summary
0,news-45155508,https://www.bbc.com/bengali/news-45155508,চীন ১০ লাখ উইগুর মুসলিমকে আটকে রেখেছে - জাতিসং...,জাতিসংঘের মানবাধিকার বিষয়ক কমিটি জেনেভাতে চীন...,আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক...,None
1,news-39127107,https://www.bbc.com/bengali/news-39127107,"কোকা-কোলা, পেপসি নিষিদ্ধ হলো তামিলনাডুতে",ভারতের দক্ষিণাঞ্চলীয় রাজ্য তামিলনাডুর ব্যবসায...,"ভারতের অন্য অঞ্চলেও কোক, পেপসি নিষিদ্ধ করার দা...",ত ামিলনাডুতে স্থানীয় পণ্যের ব্য বহার নিশ্চিত ...
2,news-49141918,https://www.bbc.com/bengali/news-49141918,ডেঙ্গু পরিস্থিতি: হাসপাতালে ভর্তির দরকার না থা...,"স্বাস্থ্য অধিদপ্তরের হিসাব মতে, গত ২৪ ঘণ্টায় ...",সোহরাওয়ার্দী হাসপাতালের চিত্র সাধারণত সেপ্টেম...,ড েঙ্গু রোগীর সংখ্যা বৃদ্ধি পেয়ে হাসপাতালগুল...
3,news-52808250,https://www.bbc.com/bengali/news-52808250,ডোনাল্ড ট্রাম্প: আমেরিকান প্রেসিডেন্টের পোস্ট ...,এই প্রথম সামাজিক মাধ্যমের বিশাল প্রতিষ্ঠান টুই...,ডোনাল্ড ট্রাম্প অভিযোগ করেছেন টু্‌ইটার আমেরিকা...,ট ুইটার ডোনাল্ড ট্রাম্পের ডাক যোগে ভোটের কাগজ ...
4,news-49047323,https://www.bbc.com/bengali/news-49047323,বাংলাদেশে বন্যার প্রকোপ: ২৪ ঘণ্টায় ডুবে যাবে ...,বাংলাদেশের বন্যা পূর্বাভাস এবং সতর্কীকরণ কেন্দ...,বন্যার প্রকোপ ছড়িয়ে পড়ছে সারা দেশে। বাংলাদে...,বা ংলাদেশের উত্তর ও মধ্যাঞ্চলে বন্যার প্রকোপ ...


In [22]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_summary'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gemini_cot_summary_results.parquet')
    start_index = df_progress['predicted_summary'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    text_body = row['text']
    
    prompt_input = chain_of_thought + "### বিবরণ: " + text_body

    try:
        df.at[i, 'predicted_summary'] = get_gemini_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gemini_cot_summary_results.parquet')

# Save final progress
df.to_parquet('gemini_cot_summary_results.parquet')

  0%|          | 0/512 [00:00<?, ?it/s]

Error at index 0: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "LOW",
      "probability_score": 0.60328925,
      "severity": "HARM_SEVERITY_HIGH",
      "severity_score": 0.83169055
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.78874594,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.42107564
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.74017435,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.5842712
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
     

In [23]:
cot_results = pd.read_parquet("gemini_cot_summary_results.parquet")
cot_results.tail()

,id,url,title,summary,text,predicted_summary
507,news-46942362,https://www.bbc.com/bengali/news-46942362,'সিরিয়ার অভ্যন্তরে ইরানি লক্ষ্যে' হামলা করছে ...,ইসরায়েলের সেনাবাহিনী জানিয়েছে যে তারা সিরিয়...,সিরিয়ার অভ্যন্তরে ইরানি লক্ষ্যবস্তুতে আঘাত কর...,ই সরায়েলি বাহিনী সিরিয়ার অভ্য ন্তরে ইরানি লক...
508,news-47308375,https://www.bbc.com/bengali/news-47308375,"আপেল-স্ট্রবেরির দরকার নেই, বাঙালিরা পেয়ারা বা...","স্থানীয়ভাবে উৎপাদিত শাকসব্জি বা ডিম, ফলমূল ইত...",পেয়ারা বাগান থেকে তোলা হচ্ছে ফসল। ফাইল ছবি জা...,"ই উনিসেফের প্রধান ইয়াসমিন আলি হকের মতে, স্থা..."
509,news-51280964,https://www.bbc.com/bengali/news-51280964,করোনাভাইরাস: চীনের ভাইরাস আক্রান্ত উহানের একাত...,করোনাভাইরাসে আক্রান্ত মানুষের সংখ্যা যখন বাড়ছ...,চীনের বাসিন্দারা ''উহান জিয়ায়ু'' বলে চিৎকার ...,"উ হানে করোনাভাইরাসের প্রাদুর্ভ াবের সময়, চীনে..."
510,news-55135935,https://www.bbc.com/bengali/news-55135935,মাদক: পুলিশের বিরুদ্ধে মাদকের সঙ্গে সংশ্লিষ্টত...,বাংলাদেশে মাদক গ্রহণের অভিযোগে সোমবার দক্ষিণ-প...,পুলিশের বিরুদ্ধে সাধারণত তিনভাবে মাদকের সঙ্গে ...,বা ংলাদেশে পুলিশ বাহিনীর মধ্যে মাদকের ব্যবহার...
511,151117_isis_terror_finance_syria_iraq_oil_kidn...,https://www.bbc.com/bengali/news/2015/11/15111...,ইসলামিক স্টেট কোথা থেকে অর্থ পায়?,ফ্রান্সের রাজধানী প্যারিসে গত শুক্রবারের আক্রম...,ইসলামিক স্টেট খেলাফতের স্বঘোষিত খলিফা আবু বকর ...,None


In [5]:
import pandas as pd
from rouge import Rouge
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_summarization(df):
    # Replace None values in 'predicted_summary' with empty strings
    df['predicted_summary'] = df['predicted_summary'].fillna("")
    
    # Filter out empty summaries
    non_empty_df = df[df['summary'].str.strip() != ""]
    non_empty_df = non_empty_df[non_empty_df['predicted_summary'].str.strip() != ""]
    
    # Initialize ROUGE
    rouge = Rouge()
    
    # Calculate ROUGE-L score
    rouge_l_scores = rouge.get_scores(non_empty_df['predicted_summary'].tolist(), non_empty_df['summary'].tolist(), avg=True)['rouge-l']
    avg_rouge_l_f = rouge_l_scores['f']
    avg_rouge_l_precision = rouge_l_scores['p']
    avg_rouge_l_recall = rouge_l_scores['r']
    
    # Calculate BERTScore
    non_empty_refs = non_empty_df['summary'].tolist()
    non_empty_preds = non_empty_df['predicted_summary'].tolist()

    if non_empty_refs and non_empty_preds:
        P, R, F1 = bert_score(non_empty_preds, non_empty_refs, lang='bn', device='cpu')
        bert_scores = {
            'bert_precision': P.mean().item(),
            'bert_recall': R.mean().item(),
            'bert_f1': F1.mean().item()
        }
    else:
        bert_scores = {
            'bert_precision': 0,
            'bert_recall': 0,
            'bert_f1': 0
        }
    
    # Calculate BLEU score
    smoothie = SmoothingFunction().method4
    bleu_scores = [sentence_bleu([ref], pred, smoothing_function=smoothie) for ref, pred in zip(non_empty_refs, non_empty_preds)]
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    
    metrics = {
        'rouge_l_f': avg_rouge_l_f,
        'rouge_l_precision': avg_rouge_l_precision,
        'rouge_l_recall': avg_rouge_l_recall,
        'bert_precision': bert_scores['bert_precision'],
        'bert_recall': bert_scores['bert_recall'],
        'bert_f1': bert_scores['bert_f1'],
        'bleu': avg_bleu_score
    }
    
    return metrics

In [6]:
zs_df = pd.read_parquet("gemini_zs_summary_results.parquet")
# zs_df['predicted_summary'] = zs_df['predicted_summary'].fillna("")
metrics = evaluate_summarization(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

rouge_l_f: 0.1098
rouge_l_precision: 0.0973
rouge_l_recall: 0.1346
bert_precision: 0.7014
bert_recall: 0.7233
bert_f1: 0.7119
bleu: 0.3141


In [7]:
ts_df = pd.read_parquet("gemini_ts_summary_results.parquet")
metrics = evaluate_summarization(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

rouge_l_f: 0.1112
rouge_l_precision: 0.1018
rouge_l_recall: 0.1316
bert_precision: 0.7033
bert_recall: 0.7218
bert_f1: 0.7121
bleu: 0.3196


In [8]:
cot_df = pd.read_parquet("gemini_cot_summary_results.parquet")
metrics = evaluate_summarization(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

rouge_l_f: 0.1121
rouge_l_precision: 0.0976
rouge_l_recall: 0.1410
bert_precision: 0.6990
bert_recall: 0.7242
bert_f1: 0.7110
bleu: 0.3148
